In [6]:
import qiskit
from qiskit import IBMQ
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

import numpy as np
from numpy import linalg
from scipy import linalg
from qiskit.quantum_info.operators import Operator, Pauli, SparsePauliOp


from qiskit import QuantumCircuit
import qiskit.quantum_info as qi

from scipy.linalg import expm
from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Kraus, SuperOp
from qiskit import Aer
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import thermal_relaxation_error
from qiskit.providers.models import BackendProperties
from qiskit.providers.aer import noise

from qiskit.quantum_info import Statevector

from qiskit.tools.monitor import job_monitor
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit import IBMQ
from qiskit.opflow import Z, I, X, Zero, One
from qiskit.circuit.library import EfficientSU2


from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

from qiskit import IBMQ, Aer
from qiskit_aer.noise import NoiseModel

from qiskit.providers.basicaer import QasmSimulatorPy  # local simulator
from qiskit.algorithms import VQE

from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

from numpy import pi, array,sqrt,exp,savetxt,loadtxt

from qiskit.providers.fake_provider import FakeManila

from qiskit.utils.mitigation import complete_meas_cal, CompleteMeasFitter
from qiskit import Aer, assemble
from qiskit.visualization import array_to_latex
from qiskit import QuantumRegister

from qiskit.opflow.state_fns    import CircuitStateFn
from qiskit.opflow.expectations import PauliExpectation
from qiskit.utils               import QuantumInstance
from qiskit.opflow              import CircuitSampler, StateFn

from scipy.optimize import minimize

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import phase_amplitude_damping_error

import matplotlib.pyplot as plt



# Initialize account.

QiskitRuntimeService.save_account(channel="ibm_quantum", token='77dda7985ecda9fa85b6dcc27eea16f88f9d96e9732aea4d184e9270ec00a8f76ec5e14f289d0a7c28ab9b755d5e205e42781d238358bc64675bcc8abbda9c1f',overwrite=True)

service = QiskitRuntimeService(channel="ibm_quantum")

backend_simulator = "ibmq_qasm_simulator"
backend = "ibmq_montreal"


In [7]:
#First we write the function for Ising hamiltonian
#We use qubits to represent sites

def H_ising(num_qubits,J,b):
    m=num_qubits-1
    n=num_qubits-2
    G=b*((I^m)^X)
    H=0
    hamiltonian1=H
    hamiltonian2=G
    
    for i in range(n+1):
        H =(I^i)^Z^Z^(I^(n-i))
        G=(I^i)^X^(I^(m-i))
        hamiltonian1=hamiltonian1+J*H
        hamiltonian2=hamiltonian2+b*G
    
    hamiltonian=hamiltonian1+hamiltonian2
    return hamiltonian

In [8]:
ops = []

J = -1
b = 4

num_qubits=4

# Create the optimization problem to solve

hamiltonian = H_ising(num_qubits,J,b)

hamiltonianOp = Operator(hamiltonian) 

In [ ]:
ansatz = EfficientSU2(num_qubits, reps=1, entanglement='linear', insert_barriers=True)

convergence = []

maxiter=5

options = Options()
options.execution.shots = 1000
options.optimization_level = 0 # No optimization
options.resilience_level = 0 # No mitigation

with Session(service=service, backend=backend_simulator, max_time=300) as session:
    estimator = Estimator(session=session, options=options)
  
    def evaluate_expectation(x):
        x = list(x)
        results = estimator.run(circuits=[ansatz], observables=[hamiltonian], parameter_values=[x]).result().values[0]
        return np.real(results)
    def callback(fx):
        convergence.append(evaluate_expectation(fx))
    
    np.random.seed(42)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    optimizer = SPSA(maxiter=maxiter,callback=callback)
    mins = optimizer.minimize(evaluate_expectation, x0=initial_point)
    

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["font.size"] = 14

# plot convergence value
plt.figure(figsize=(12, 6))
plt.plot(convergence, label="Estimator VQE")
plt.axhline(y=real_solution, color="tab:red", ls="--", label="Target")

plt.legend(loc="best")
plt.xlabel("Iteration")
plt.ylabel("Energy [H]")
plt.title("VQE energy")
plt.show()

In [ ]:
options = Options()
options.execution.shots = 1000
options.optimization_level = 0 # No optimization
options.resilience_level = 0 # No error mitigation

with Session(service=service, backend=backend, max_time=300) as session:
    estimator = Estimator(session=session, options=options)
  
    def evaluate_expectation(x):
        x = list(x)
        results = estimator.run(circuits=[ansatz], observables=[hamiltonian], parameter_values=[x]).result().values[0]
        return np.real(results)
    def callback(fx):
        convergence.append(evaluate_expectation(fx))
    
    np.random.seed(42)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    mins = minimize(evaluate_expectation, x0=initial_point,method='cobyla', bounds=None,callback=callback)

In [ ]:
options = Options()
options.resilience_level = 1 # T-REx
options.optimization_level = 0 # No optimization
options.execution.shots = 1000

with Session(service=service, backend=backend, max_time=300) as session:
    estimator = Estimator(session=session, options=options)
  
    def evaluate_expectation(x):
        x = list(x)
        results = estimator.run(circuits=[ansatz], observables=[hamiltonian], parameter_values=[x]).result().values[0]
        return np.real(results)
    def callback(fx):
        convergence.append(evaluate_expectation(fx))
    
    np.random.seed(42)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    mins = minimize(evaluate_expectation, x0=initial_point,method='cobyla', bounds=None,callback=callback)
    
    

In [ ]:
options = Options()
options.execution.shots = 1000
options.optimization_level = 0 # No optimization
options.resilience_level = 2 # ZNE

with Session(service=service, backend=backend, max_time=300) as session:
    estimator = Estimator(session=session, options=options)
  
    def evaluate_expectation(x):
        x = list(x)
        results = estimator.run(circuits=[ansatz], observables=[hamiltonian], parameter_values=[x]).result().values[0]
        return np.real(results)
    def callback(fx):
        convergence.append(evaluate_expectation(fx))
    
    np.random.seed(42)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    mins = minimize(evaluate_expectation, x0=initial_point,method='cobyla', bounds=None,callback=callback)
    
    

In [ ]:
def interim_results_callback(job_id, result):
    now = datetime.datetime.now()
    print(now, "*** Callback ***", result, "\n")

options = Options()
options.optimization_level = 0 # No optimization
options.execution.shots = 100
options.resilience_level = 3 # PEC
options.environment.callback = interim_results_callback

with Session(service=service, backend=backend, max_time=300) as session:
    estimator = Estimator(session=session, options=options)
  
    def evaluate_expectation(x):
        x = list(x)
        results = estimator.run(circuits=[ansatz], observables=[hamiltonian], parameter_values=[x]).result().values[0]
        return np.real(results)
    def callback(fx):
        convergence.append(evaluate_expectation(fx))
    
    np.random.seed(42)
    initial_point = np.random.uniform(-np.pi, np.pi, ansatz.num_parameters)
    mins = minimize(evaluate_expectation, x0=initial_point,method='cobyla', bounds=None,callback=callback)